# VGGNET


# 라이브러리 불러오기

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"device = {device}")
torch.manual_seed(123)
if device == 'cuda':
    torch.cuda.manual_seed_all(123)

device = cuda


# 데이터


In [5]:
# 하이퍼 파라미터 설정
lr = 0.001
epoch = 20
batch_size = 100

In [6]:
# MNIST 데이터셋 불러오기
mnist_train = dset.MNIST(root='MNIST_data/',
                          train=True, # 트레인 셋 
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dset.MNIST(root='MNIST_data/',
                         train=False, # 테스트 셋
                         transform=transforms.ToTensor(),
                         download=True)


Extracting MNIST_data/MNIST/raw/train-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/train-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-images-idx3-ubyte.gz to MNIST_data/MNIST/raw




Extracting MNIST_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to MNIST_data/MNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [7]:
# 데이터 로더
data_loader = torch.utils.data.DataLoader(dataset=mnist_train, # 훈련용 데이터 로딩
                                          batch_size=batch_size,
                                          shuffle=True, # 에폭마다 데이터 섞기
                                          drop_last=True)

# 모델

In [8]:
# CNN 
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)  
        out = self.fc(out)
        return out

In [9]:
model = CNN().to(device)

In [13]:
criterion = torch.nn.CrossEntropyLoss().to(device) 
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [15]:
# 훈련
total_batch = len(data_loader)
print('Training started.')
for epoch in range(epoch):
    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))

print('Model train is finished')

Training started.
[Epoch:    1] cost = 0.216502905
[Epoch:    2] cost = 0.0646788701
[Epoch:    3] cost = 0.0472199023
[Epoch:    4] cost = 0.0373744071
[Epoch:    5] cost = 0.0318305083
[Epoch:    6] cost = 0.0260258708
[Epoch:    7] cost = 0.0225484483
[Epoch:    8] cost = 0.0194502901
[Epoch:    9] cost = 0.0158155337
[Epoch:   10] cost = 0.0133337202
[Epoch:   11] cost = 0.0118209561
[Epoch:   12] cost = 0.0102256602
[Epoch:   13] cost = 0.00959352497
[Epoch:   14] cost = 0.0072978856
[Epoch:   15] cost = 0.00632689055
[Epoch:   16] cost = 0.00590851251
[Epoch:   17] cost = 0.00472865393
[Epoch:   18] cost = 0.00479088724
[Epoch:   19] cost = 0.00426766416
[Epoch:   20] cost = 0.00488755247
Model train is finished


In [16]:
with torch.no_grad():
    X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.test_labels.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())

Accuracy: 0.9876999855041504


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:69: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:59: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")
